In [1]:
%load_ext jupyter_black

In [2]:
import json
from pathlib import Path
from shutil import copyfileobj

import pandas as pd
from requests import Session, HTTPError

In [8]:
template = "https://mtarchive.geol.iastate.edu/%Y/%m/%d/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_%Y%m%d_%H%M00.json"
outdir = Path("/home/leaver2000/sppp/data/probsevere/2022/20")


def download_archive_data(
    start: str,
    end: str,
    outdir: Path,
    freq: str = "2min",
) -> None:
    urls = pd.date_range(start=start, end=end, freq=freq).strftime(template)
    # print(urls)
    with Session() as session:
        for url in urls:
            try:
                r = session.get(url, stream=True)
                r.raise_for_status()
                outfile = outdir / r.url.split("/")[-1]

                with outfile.open("w") as f:
                    json.dump(r.json(), f, indent=4)

            except (ConnectionError, HTTPError):
                print("error downloading", url)


if __name__ == "__main__":
    download_archive_data(
        start="2022-07-20T00:00:00Z", end="2022-07-21T00:00:00Z", outdir=outdir
    )

error downloading https://mtarchive.geol.iastate.edu/2022/07/20/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_20220720_225600.json
error downloading https://mtarchive.geol.iastate.edu/2022/07/20/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_20220720_225800.json
error downloading https://mtarchive.geol.iastate.edu/2022/07/20/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_20220720_230200.json
error downloading https://mtarchive.geol.iastate.edu/2022/07/20/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_20220720_230400.json
